# 机械臂手眼标定-眼在手上-Python实现
## 导入依赖

In [1]:
import numpy as np
# 自定义库
from transform import Transform
from pose import Pose
from tsai import calibration_eye_in_hand

np.set_printoptions(precision=3, suppress=True)

## 原始数据

In [2]:
# 原始数据: 腕关节在机械臂基坐标系下的位姿 ^{arm}_{wrist}T
# 数据来源: 通过示教器读取
# 数据格式: x,y,z，roll,pitch,yaw
# xyz为三维坐标，单位m
# [roll, pitch, yaw] 为欧拉角, 单位°
raw_wrist_pose = np.float64([
    [0.82772, 0.06522, 0.75224, -144.93, 67.32, -145.78],
    [0.80865, 0.12002, 0.80706, 174.46, 58.62, 158.47],
    [0.80866, 0.12544, 0.67755, 110.68, 83.35, 89.47],
    [0.82286, 0.05224, 0.66590, -87.14, 70.66, -79.66],
    [0.88218,-0.01751, 0.75942, -131.85,60.81, -127.65],
    [0.87886, 0.02781, 0.80272, 148.30, 53.81, 152.58],
    [0.93914, 0.15168, 0.76964, 175.39, 66.25, 163.54],
    [0.92465, 0.19866, 0.72486, -152.84,75.11, -171.24],
    [0.91312, 0.14485, 0.72482, -106.60, 55.06, -124.55],
    [0.91310, 0.05494, 0.72477, -176.14, 74.58, -167.83],
    [0.92506, 0.08207, 0.80342, -129.92, 49.88, -136.51],
    [0.93684, 0.12112, 0.72428, 148.61, 71.30, 141.06]])

# 原始数据: 标定板在相机坐标系下的位姿 ^{cami}_{board}T
# 数据来源: 相机标定得到，相机外参。
# 数据格式: x,y,z,xn,yn,zn 
# xyz为三维坐标，单位m
# [xn, yn, zn]=rvect 旋转向量, 旋转角度为旋转向量的模长.
raw_caliboard_pose = np.float64([
    [-0.05278, -0.08160, 0.39593,  0.02834, -0.02854,  0.22926],
    [-0.13787, -0.07793, 0.37583,  0.24938, -0.22099, -0.15542],
    [-0.17817, -0.00343, 0.40412, -0.26167, -0.32992, -0.16666],
    [ 0.00962, -0.02616, 0.45339, -0.33369,  0.08834,  0.35973],
    [-0.08343, -0.09897, 0.34839,  0.03381,  0.03085,  0.40624],
    [-0.10354, -0.03125, 0.34939,  0.26487,  0.14562, -0.27929],
    [-0.04634, -0.02492, 0.27151,  0.12825, -0.16397, -0.09373],
    [-0.02279, -0.03672, 0.29697, -0.02907, -0.30641,  0.03926],
    [-0.01490, -0.06452, 0.29502, -0.01287, -0.36766,  0.50575],
    [-0.03314, -0.04779, 0.32411, -0.01859,  0.15740,  0.07058],
    [-0.04223, -0.06639, 0.27773,  0.19789, -0.16627,  0.49610],
    [-0.07012, -0.02279, 0.28054, -0.01190, -0.07518, -0.19488]])



## 构造齐次变换矩阵

In [3]:
print("T_arm2wrist_list")
# 腕关节在机械臂基坐标系下的位姿
T_arm2wrist_list = []
for x, y, z, roll, pitch, yaw in raw_wrist_pose:
    # 欧拉角转换为弧度
    rpy = np.radians(np.float64([roll, pitch, yaw]))
    # 创建位姿对象
    pose = Pose()
    pose.set_position(x, y, z) # 设置位置
    pose.set_euler_angle(*rpy) # 设置欧拉角
    # 获取齐次变换矩阵
    T_arm2wristi = pose.get_transform_matrix()
    print(T_arm2wristi)
    T_arm2wrist_list.append(T_arm2wristi)

T_arm2wrist_list
[[-0.319 -0.022  0.948  0.828]
 [-0.217  0.975 -0.05   0.065]
 [-0.923 -0.222 -0.316  0.752]
 [ 0.     0.     0.     1.   ]]
[[-0.484  0.289  0.826  0.809]
 [ 0.191  0.956 -0.222  0.12 ]
 [-0.854  0.05  -0.518  0.807]
 [ 0.     0.     0.     1.   ]]
[[ 0.001  0.362  0.932  0.809]
 [ 0.116  0.926 -0.359  0.125]
 [-0.993  0.108 -0.041  0.678]
 [ 0.     0.     0.     1.   ]]
[[ 0.059 -0.12   0.991  0.823]
 [-0.326  0.936  0.133  0.052]
 [-0.944 -0.331  0.017  0.666]
 [ 0.     0.     0.     1.   ]]
[[-0.298 -0.131  0.946  0.882]
 [-0.386  0.922  0.006 -0.018]
 [-0.873 -0.363 -0.325  0.759]
 [ 0.     0.     0.     1.   ]]
[[-0.524  0.015  0.852  0.879]
 [ 0.272  0.951  0.15   0.028]
 [-0.807  0.31  -0.502  0.803]
 [ 0.     0.     0.     1.   ]]
[[-0.386  0.212  0.898  0.939]
 [ 0.114  0.977 -0.181  0.152]
 [-0.915  0.032 -0.401  0.77 ]
 [ 0.     0.     0.     1.   ]]
[[-0.254  0.3    0.919  0.925]
 [-0.039  0.947 -0.32   0.199]
 [-0.966 -0.117 -0.229  0.725]
 [ 0.     0.   

In [4]:
print("T_cam2board_list")
T_cam2board_list = []
for x, y, z, xn, yn, zn in raw_caliboard_pose:
    # 创建位姿对象
    pose = Pose()
    pose.set_position(x, y, z)
    rmat = Transform.rvect2rmat([xn, yn, zn])
    pose.set_rotation_matrix(rmat)
    # 获取齐次变换矩阵
    T_cami2board = pose.get_transform_matrix()
    print(T_cami2board)
    T_cam2board_list.append(T_cami2board)

T_cam2board_list
[[ 0.973 -0.228 -0.025 -0.053]
 [ 0.227  0.973 -0.031 -0.082]
 [ 0.032  0.025  0.999  0.396]
 [ 0.     0.     0.     1.   ]]
[[ 0.964  0.125 -0.235 -0.138]
 [-0.179  0.957 -0.227 -0.078]
 [ 0.197  0.261  0.945  0.376]
 [ 0.     0.     0.     1.   ]]
[[ 0.933  0.203 -0.297 -0.178]
 [-0.119  0.953  0.28  -0.003]
 [ 0.34  -0.226  0.913  0.404]
 [ 0.     0.     0.     1.   ]]
[[ 0.933 -0.359  0.026  0.01 ]
 [ 0.331  0.882  0.336 -0.026]
 [-0.144 -0.304  0.942  0.453]
 [ 0.     0.     0.     1.   ]]
[[ 0.918 -0.395  0.037 -0.083]
 [ 0.396  0.918 -0.027 -0.099]
 [-0.023  0.039  0.999  0.348]
 [ 0.     0.     0.     1.   ]]
[[ 0.951  0.29   0.105 -0.104]
 [-0.252  0.927 -0.278 -0.031]
 [-0.178  0.237  0.955  0.349]
 [ 0.     0.     0.     1.   ]]
[[ 0.982  0.082 -0.169 -0.046]
 [-0.103  0.987 -0.119 -0.025]
 [ 0.157  0.135  0.978  0.272]
 [ 0.     0.     0.     1.   ]]
[[ 0.953 -0.034 -0.302 -0.023]
 [ 0.043  0.999  0.023 -0.037]
 [ 0.301 -0.035  0.953  0.297]
 [ 0.     0.   

## 手眼标定

In [5]:
# 标定结果输出的是
# 腕关节坐标系 -> 相机坐标系的空间变换
T_wrist2cam = calibration_eye_in_hand(T_arm2wrist_list, T_cam2board_list)
print("T_wrist2cam")
print(T_wrist2cam)

[[ 0.    -0.755  0.398]
 [ 0.755  0.     0.076]
 [-0.398 -0.076  0.   ]]
N_Ai: [0.169 0.238 0.377]
N_Bi: [-0.246  0.16   0.378]
N_Ai + N_Bi: [-0.076  0.398  0.755]
sigma: [0.856 0.856 0.   ]
Rank Skew(N_Ai + N_Bi): 2
[[ 0.    -0.786  0.069]
 [ 0.786  0.    -0.571]
 [-0.069  0.571  0.   ]]
N_Ai: [ 0.325 -0.256  0.391]
N_Bi: [0.245 0.324 0.395]
N_Ai + N_Bi: [0.571 0.069 0.786]
sigma: [0.974 0.974 0.   ]
Rank Skew(N_Ai + N_Bi): 2
[[ 0.     0.256 -0.445]
 [-0.256  0.    -0.288]
 [ 0.445  0.288  0.   ]]
N_Ai: [-0.082 -0.373 -0.128]
N_Bi: [ 0.37  -0.072 -0.128]
N_Ai + N_Bi: [ 0.288 -0.445 -0.256]
sigma: [0.589 0.589 0.   ]
Rank Skew(N_Ai + N_Bi): 2
[[ 0.     0.355 -0.064]
 [-0.355  0.     0.053]
 [ 0.064 -0.053  0.   ]]
N_Ai: [-0.057 -0.008 -0.178]
N_Bi: [ 0.004 -0.056 -0.178]
N_Ai + N_Bi: [-0.053 -0.064 -0.355]
sigma: [0.365 0.365 0.   ]
Rank Skew(N_Ai + N_Bi): 2
[[ 0.    -0.991  0.011]
 [ 0.991  0.     0.414]
 [-0.011 -0.414  0.   ]]
N_Ai: [-0.193  0.217  0.497]
N_Bi: [-0.221 -0.205  0.494

## 验证标定结果

标定板相对于机械臂是静止不动的， 因此通过新求得的`T_wrist2cam`, 来计算标定板在机械臂基坐标系下的位姿$^{arm}_{board}T$
然后对比所有的采样数据，看标定板原点在机械臂基坐标系下坐标的分布情况。

$$
^{arm}_{board}T = ^{arm}_{wrist}T * ^{wrist}_{cam}T * ^{cam}_{board}T 
$$

In [6]:
n_sample = len(T_arm2wrist_list)
t_arm2board_list = []
for i in range(n_sample):
    T_arm2wristi = T_arm2wrist_list[i]
    T_cami2board = T_cam2board_list[i]
    T_arm2board = np.dot(np.dot(T_arm2wristi, T_wrist2cam), T_cami2board)
    t_arm2board = T_arm2board[:3, 3]
    print(f"标定板坐标[{i}]: {t_arm2board}")
    t_arm2board_list.append(t_arm2board)

标定板坐标[0]: [1.237 0.172 0.79 ]
标定板坐标[1]: [1.237 0.172 0.791]
标定板坐标[2]: [1.237 0.173 0.792]
标定板坐标[3]: [1.235 0.174 0.791]
标定板坐标[4]: [1.234 0.173 0.791]
标定板坐标[5]: [1.228 0.169 0.794]
标定板坐标[6]: [1.228 0.17  0.797]
标定板坐标[7]: [1.227 0.171 0.798]
标定板坐标[8]: [1.228 0.172 0.797]
标定板坐标[9]: [1.228 0.171 0.795]
标定板坐标[10]: [1.229 0.171 0.796]
标定板坐标[11]: [1.228 0.169 0.797]


In [7]:
t_arm2board_list = np.float64(t_arm2board_list)
# 计算均值
t_arm2board_mean = np.mean(t_arm2board_list, axis=0)
print(f"标定板在机械臂基坐标系下的位置(均值):\n {t_arm2board_mean}")
error_xyz = t_arm2board_list - t_arm2board_mean
print(f"各轴误差: error_x, error_y, error_z \n{error_xyz}")
distance = np.linalg.norm(error_xyz, axis=1)
print(f"欧式距离: {distance}")
distance_mean = np.mean(distance)
print(f"distance_mean : {distance_mean}")

标定板在机械臂基坐标系下的位置(均值):
 [1.231 0.171 0.794]
各轴误差: error_x, error_y, error_z 
[[ 0.006  0.001 -0.004]
 [ 0.005  0.    -0.003]
 [ 0.005  0.001 -0.002]
 [ 0.004  0.003 -0.003]
 [ 0.002  0.002 -0.003]
 [-0.003 -0.003  0.   ]
 [-0.004 -0.002  0.003]
 [-0.004 -0.     0.004]
 [-0.003  0.001  0.003]
 [-0.003 -0.001  0.001]
 [-0.003 -0.     0.002]
 [-0.003 -0.002  0.003]]
欧式距离: [0.007 0.006 0.006 0.005 0.004 0.004 0.005 0.005 0.004 0.003 0.003 0.005]
distance_mean : 0.004824784748317957
